**1. 준비 (GPU T4로 세팅)**

In [2]:
!nvidia-smi -L || echo "No GPU detected (ok for quick tests)"
!pip -q install "transformers==4.44.0" "datasets==3.0.1" "torch" "sentencepiece" \
                 "pandas" "numpy" "matplotlib" "tqdm" "evaluate"

/bin/bash: line 1: nvidia-smi: command not found
No GPU detected (ok for quick tests)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 927.6 kB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_ve

**2. Github Link 추가하여 데이터 불러오기, 데이터 파일 명만 추가하여 파일 구분**

In [4]:
import pandas as pd

# GitHub raw URL 목록
urls = [
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/1.R1%7E2.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/2.RTM.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/3.R3%7E5.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/4.PLAY-IN.csv",
    "https://raw.githubusercontent.com/hyunseo-adastra/SocialData_SportsCommunity/refs/heads/main/LCK_data/5.PLAYOFF.csv"
]

dfs = []

for url in urls:
    # 파일 이름만 추출 (예: Round1_Bahrain.csv → Round1_Bahrain)
    filename = url.split("/")[-1].replace(".csv", "")

    # CSV 읽기
    df = pd.read_csv(url)

    # 파일 이름 컬럼 추가
    df["source_file"] = filename

    dfs.append(df)

# 모든 데이터 합치기
df_all = pd.concat(dfs, ignore_index=True)
df_all.head()

,time_text,timestamp,author,message,amount,format,date,source_file
0,-1:54:31,1749014450918319,UCpTHr9Rn5_w9BEMheofv82w,KT 파이팅!!!,NaN,R1~2,250604,1.R1%7E2
1,-1:42:23,1749015179089550,UCuSabTEvRG-f7w6Pa80RyLg,쇼메이커는 무조건 1찍이다,NaN,R1~2,250604,1.R1%7E2
2,-1:32:52,1749015750551203,UCNInwRrJCCrVFBz-kkxVqfA,블루가어디임?,NaN,R1~2,250604,1.R1%7E2
3,-1:25:43,1749016179287640,UCC4cqWCL8LGCsQLmB8afheA,T,NaN,R1~2,250604,1.R1%7E2
4,-1:25:25,1749016197284369,UC13UxJwYop0g_YWB-wA08Uw,ㄷㄱㄷㄱ 아무나 이겨랑,NaN,R1~2,250604,1.R1%7E2


**3. 데이터 컬럼 선택 및 클리닝**

클리닝
- 채팅 제한 봇 데이터 삭제 (유저id: UCSvjQBDgYDB5TGVmCZObcwA)
- 디시인사이드의 경우 "dc official App" 삭제
- 해시태그, 멘션, URL, 공백 삭제
- "ㅋㅋㅋㅋㅋ"나 "!!!!!" 등의 표현 수 두 글자로 축약

**추후논의하여 클리닝 수정 가능

In [9]:
# 마이크로초 단위 timestamp를 datetime 객체로 변환
# unit='us'를 사용하여 마이크로초 단위임을 지정합니다.
df_all['datetime_utc'] = pd.to_datetime(df_all['timestamp'], unit='us', errors='coerce')

# 초 이하 단위 제거
df_all['datetime_utc'] = df_all['datetime_utc'].dt.floor('s')

# UTC 시간을 KST (UTC+9)로 변환
# 먼저 UTC 타임존 정보를 추가하고, 그 다음 KST로 변환합니다.
df_all['datetime_kst'] = df_all['datetime_utc'].dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')

# 초 이하 단위 제거
df_all['datetime_kst'] = df_all['datetime_kst'].dt.floor('s')

In [10]:
import pandas as pd
import numpy as np
import re

# ---------------------------------------------------------------------
# 데이터 클리닝

def normalize_text(s: str) -> str:
    if not isinstance(s, str):
        return ""

    # 1) URL 제거
    s = re.sub(r"http\S+|www\.\S+", " ", s)

    # 2) 멘션/해시태그 제거
    s = re.sub(r"[@#]\w+", " ", s)

    # 3) 반복 문자 축약 (ㅋㅋㅋㅋ -> ㅋㅋ, 아아아아 -> 아아, !!!!! -> !!)
    s = re.sub(r"([ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9!?.])\1{2,}", r"\1\1", s)

    # 생략 4) 이모지/특수문자 과도한 것 정리 (한글/영문/숫자/기본문장부호만 유지)
    # s = re.sub(r"[^\w\s가-힣ㄱ-ㅎㅏ-ㅣ!?.]", " ", s, flags=re.UNICODE)

    # 생략 5) 팀명/드라이버 약칭 정규화 (원하면 계속 추가)
    # team_map = {
    #    "T1": "티원",
    #    "GEN": "젠지",
    #    "VER": "페르스타펜",
    #    "HAM": "해밀턴",
    #}
    #for short_name, full_name in team_map.items():
    #    s = re.sub(fr"\b{short_name}\b", full_name, s, flags=re.IGNORECASE)

    # 6) 공백 정리
    s = re.sub(r"\s+", " ", s).strip()
    return s

####클리닝은 논의 후 추가

# ---------------------------------------------------------------------
# 데이터프레임 컬럼 선택 (df_all에서 필요한 컬럼 선택)
df = df_all[["datetime_kst", "author", "message", "format"]].copy()

# 채팅 제한 봇 데이터 삭제 (유저id: UCSvjQBDgYDB5TGVmCZObcwA)
df = df[df['author'] != 'UCSvjQBDgYDB5TGVmCZObcwA'].copy()

# 클리닝
df["clean_text"] = df["message"].map(normalize_text)


# 빈 텍스트/공백만 남은 행 제거
df["clean_text"] = df["clean_text"].fillna("").str.strip()
df = df[df["clean_text"].astype(bool)].reset_index(drop=True)

print("After cleaning, rows:", len(df))


After cleaning, rows: 1505672


In [11]:
df.head()

,datetime_kst,author,message,format,clean_text
0,2025-06-04 14:20:50+09:00,UCpTHr9Rn5_w9BEMheofv82w,KT 파이팅!!!,R1~2,KT 파이팅!!
1,2025-06-04 14:32:59+09:00,UCuSabTEvRG-f7w6Pa80RyLg,쇼메이커는 무조건 1찍이다,R1~2,쇼메이커는 무조건 1찍이다
2,2025-06-04 14:42:30+09:00,UCNInwRrJCCrVFBz-kkxVqfA,블루가어디임?,R1~2,블루가어디임?
3,2025-06-04 14:49:39+09:00,UCC4cqWCL8LGCsQLmB8afheA,T,R1~2,T
4,2025-06-04 14:49:57+09:00,UC13UxJwYop0g_YWB-wA08Uw,ㄷㄱㄷㄱ 아무나 이겨랑,R1~2,ㄷㄱㄷㄱ 아무나 이겨랑


**4. 파인튜닝을 위한 데이터셋 불러오기**

사용데이터: NSMC, Steam
- NSMC 네이버 영화 댓글: https://github.com/e9t/nsmc
- Steam 리뷰: https://github.com/bab2min/corpus

In [ ]:
from datasets import load_dataset, DatasetDict, concatenate_datasets
import pandas as pd

# NSMC (네이버 영화리뷰) 불러오기
nsmc = load_dataset(
    "csv",
    data_files={
        "train": "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
        "test":  "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",
    },
    delimiter="\t"
)
print("NSMC loaded:", nsmc)

# Steam 한국어 리뷰 데이터 불러오기
# 파일 구조: 각 줄이 "label\ttext" 형태
steam = load_dataset(
    "csv",
    data_files={"train": "https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/steam.txt"},
    delimiter="\t",
    column_names=["label", "text"]
)
print("Steam dataset loaded:", steam)

# 컬럼 구조 통일
# NSMC는 "document" 컬럼 → text로 이름 변경
nsmc = nsmc.rename_column("document", "text")

# 라벨 타입 통일 (float → int 등)
nsmc["train"] = nsmc["train"].map(lambda x: {"label": int(x["label"])})
nsmc["test"]  = nsmc["test"].map(lambda x: {"label": int(x["label"])})
steam["train"] = steam["train"].map(lambda x: {"label": int(x["label"])})

# Steam train 데이터와 NSMC train/test 합치기
train_merged = concatenate_datasets([nsmc["train"], steam["train"]])
test_merged  = nsmc["test"]  # Steam에는 test 세트가 없으므로 그대로 사용

merged = DatasetDict({"train": train_merged, "test": test_merged})
print("통합 DatasetDict 구성 완료:")
print(merged)

NSMC loaded: DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})
Steam dataset loaded: DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 100000
    })
})
통합 DatasetDict 구성 완료:
DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 250000
    })
    test: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 50000
    })
})


**5. KcElectra 모델 불러오기, Tokenizing**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
import evaluate
import torch
import numpy as np

# 모델 후보. 첫 번째가 잘 불리면 그대로 사용.
BASES = ["beomi/KcELECTRA-base", "beomi/KcELECTRA-base-v2022"]

tok = None
MODEL_BASE = None
for base in BASES:
    try:
        tok = AutoTokenizer.from_pretrained(base)
        MODEL_BASE = base
        print("Using base:", base)
        break
    except Exception as e:
        print("Tokenizer load failed:", base, "->", e)

if tok is None:
    raise RuntimeError("KcELECTRA 토크나이저 로드 실패")

def tok_fn(batch):
    # Use the "text" column for tokenization as it's available in the merged dataset
    texts = [str(text) for text in batch["text"]]
    out = tok(texts, truncation=True, max_length=128)
    # Add the label column
    out["labels"] = batch["label"]
    return out


# nsmc 데이터셋 대신 클리닝된 df 데이터프레임을 사용
# 데이터프레임을 Dataset 객체로 변환
from datasets import Dataset
# df_dataset = Dataset.from_pandas(df) # This is not used for training anymore

# tokenized = df_dataset.map(
#     tok_fn,
#     batched=True,
#     # remove_columns는 더 이상 nsmc 데이터셋을 사용하지 않으므로 필요 없습니다.
#     # remove_columns=remove_cols
# )


collator = DataCollatorWithPadding(tokenizer=tok)
metric_f1 = evaluate.load("f1")

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_BASE,
    num_labels=2 # 이진 분류 (긍정/부정)를 위한 num_labels=2
)

print("Tokenized dataset structure:", merged) # Print the merged dataset structure

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Using base: beomi/KcELECTRA-base


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenized dataset structure: DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 250000
    })
    test: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 50000
    })
})


**6. KcElectra Model Training(Finetuning)**

In [12]:
from transformers import TrainingArguments, Trainer

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir="./results",          # 훈련 결과 및 체크포인트 저장 디렉토리
    evaluation_strategy="epoch",     # 에포크마다 평가 수행
    save_strategy="epoch",           # 에포크마다 모델 저장
    learning_rate=2e-5,              # 학습률
    per_device_train_batch_size=16,  # 장치당 훈련 배치 크기
    per_device_eval_batch_size=16,   # 장치당 평가 배치 크기
    num_train_epochs=3,              # 훈련 에포크 수 (빠른 테스트를 위해 1로 줄임)
    weight_decay=0.01,               # 가중치 감소 (L2 정규화)
    push_to_hub=False,               # Hugging Face Hub에 푸시하지 않음
    report_to="none",                # 로깅 비활성화
    warmup_steps=500,                # 추가: 학습 초반 warmup
    logging_steps=500,               # 추가: 500 스텝마다 로그
    load_best_model_at_end=True,     # 추가: 가장 좋은 모델 로드
    metric_for_best_model="f1",      # 추가: F1 기준으로 최고 모델 선택
)

# 평가 메트릭 함수 정의
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric_f1.compute(predictions=predictions, references=labels)

# Trainer 인스턴스 생성
# 훈련 데이터셋은 수동으로 토큰화한 tokenized_inputs 리스트를 사용합니다.
# 검증 데이터셋은 NSMC 테스트 데이터셋을 사용하여 토큰화해야 합니다.

# NSMC 테스트 데이터셋 토큰화
test_texts = [str(text) for text in nsmc["test"]["text"]]
test_labels = nsmc["test"]["label"]

# The previous tokenization logic was for `df_dataset` which is not the training data.
# The training data should be `merged["train"]` and the evaluation data `merged["test"]`.
# Also, the tokenization should be done using the `map` function of the Dataset object, not manually in a loop.

# Tokenize the merged datasets
tokenized_merged = merged.map(
    tok_fn,
    batched=True,
    remove_columns=["id", "text"] # Remove original text columns after tokenization
)

collator = DataCollatorWithPadding(tokenizer=tok)
metric_f1 = evaluate.load("f1")

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_BASE,
    num_labels=2 # 이진 분류 (긍정/부정)를 위한 num_labels=2
)

trainer = Trainer(
    model=model,                         # 훈련할 모델
    args=training_args,                  # 훈련 인자
    train_dataset=tokenized_merged["train"],         # 훈련 데이터셋
    eval_dataset=tokenized_merged["test"],           # 검증 데이터셋
    data_collator=collator,              # 데이터 콜레이터
    compute_metrics=compute_metrics,     # 평가 메트릭 함수
    tokenizer=tok                        # 토크나이저
)

# 훈련 시작
print("Starting training...")
trainer.train()
print("Training finished.")

KeyboardInterrupt: 

**7. 감성분석**

In [ ]:
import torch, numpy as np
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os
import glob

# MODEL_PATH를 훈련 결과가 저장된 디렉토리로 변경
# './results' 디렉토리에 저장된 최신 체크포인트를 로드합니다.
TRAINING_OUTPUT_DIR = "./results"
BASE = "beomi/KcELECTRA-base"                # 토크나이저용 베이스

# 가장 최근 체크포인트 디렉토리를 찾습니다.
checkpoints = [d for d in glob.glob(f"{TRAINING_OUTPUT_DIR}/checkpoint-*") if os.path.isdir(d)]
if not checkpoints:
    raise FileNotFoundError(f"No checkpoint directories found in {TRAINING_OUTPUT_DIR}")

# 체크포인트 이름에서 숫자를 추출하여 최신 체크포인트를 찾습니다.
latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[-1]))
MODEL_PATH = latest_checkpoint
print(f"Loading model from: {MODEL_PATH}")


tok = AutoTokenizer.from_pretrained(BASE)
# 훈련된 모델 로드
clf = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).eval().to("cuda" if torch.cuda.is_available() else "cpu")
device = "cuda" if torch.cuda.is_available() else "cpu"

def batched(lst, n=64):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

probs = []
for chunk in tqdm(batched(df["clean_text"].tolist(), 64), total=(len(df)//64 + 1)):
    enc = tok(chunk, truncation=True, padding=True, max_length=128, return_tensors="pt")
    enc = {k:v.to(device) for k,v in enc.items()}
    with torch.no_grad():
        p = clf(**enc).logits.softmax(dim=-1).cpu().numpy()  # [:,0]=neg, [:,1]=pos
    probs.append(p)

probs = np.vstack(probs)
df["p_neg"], df["p_pos"] = probs[:,0], probs[:,1]
df["polarity_binary"] = (df["p_pos"] > df["p_neg"]).astype(int)  # 1=pos, 0=neg

# 휴리스틱 중립(신뢰도 낮은 샘플)
conf = np.abs(df["p_pos"] - 0.5) * 2  # 0..1
NEUTRAL_THRESH = 0.20                 # 필요시 0.15~0.30에서 튜닝
df["neutral_flag"] = (conf < NEUTRAL_THRESH).astype(int)

def to_ternary(row):
    if row["neutral_flag"] == 1: return 1  # neutral
    return 2 if row["polarity_binary"]==1 else 0  # 2=pos, 0=neg

df["polarity_3cls"] = df.apply(to_ternary, axis=1)
df[["clean_text","p_pos","p_neg","polarity_binary","polarity_3cls"]].head(10)